In [89]:
# Python program to print a given number in
# words. The program handles numbers
# from 0 to 9999
  
  
def convert_to_words(num):
    result = ""
  
    # Get number of digits
    # in given number
    l = len(num)
  
    # Base cases
    if (l == 0):
        return "empty string"
  
    if (l > 4):
        return "Length more than 4 is not supported"
  
    # The first string is not used,
    # it is to make array indexing simple
    single_digits = ["zero", "one", "two", "three",
                     "four", "five", "six", "seven",
                     "eight", "nine"]
  
    # The first string is not used,
    # it is to make array indexing simple
    two_digits = ["", "ten", "eleven", "twelve",
                  "thirteen", "fourteen", "fifteen",
                  "sixteen", "seventeen", "eighteen",
                  "nineteen"]
  
    # The first two string are not used,
    # they are to make array indexing simple
    tens_multiple = ["", "", "twenty", "thirty", "forty",
                     "fifty", "sixty", "seventy", "eighty",
                     "ninety"]
  
    tens_power = ["hundred", "thousand"]
  
    # For single digit number
    if (l == 1):
        return (single_digits[ord(num[0]) - 48])
  
    # Iterate while num is not '\0'
    x = 0
    while (x < len(num)):
  
        # Code path for first 2 digits
        if (l >= 3):
            if (ord(num[x]) - 48 != 0):
               result += single_digits[ord(num[x]) - 48] + " "
               result += tens_power[l - 3] + " "
                # here len can be 3 or 4
  
            l -= 1
  
        # Code path for last 2 digits
        else:
  
            # Need to explicitly handle
            # 10-19. Sum of the two digits
            # is used as index of "two_digits"
            # array of strings
            if (ord(num[x]) - 48 == 1):
                sum = (ord(num[x]) - 48 +
                       ord(num[x+1]) - 48)
                result += two_digits[sum]
                return result
  
            # Need to explicitly handle 20
            elif (ord(num[x]) - 48 == 2 and
                  ord(num[x + 1]) - 48 == 0):
                result += "twenty"
                return result
  
            # Rest of the two digit
            # numbers i.e., 21 to 99
            else:
                i = ord(num[x]) - 48
                if(i > 0):
                    result += tens_multiple[i] + " "
                x += 1
                if(ord(num[x]) - 48 != 0):
                    result += single_digits[ord(num[x]) - 48]
        x += 1
    return result
  
# This code is contributed
# by Mithun Kumar

print(convert_to_words("1234"))

one thousand two hundred thirty four


In [26]:
!pip3 install torch

In [90]:
import torch
import torch.nn.functional as F
from torch import nn

# define the network class
class Network(nn.Module):
    def __init__(self):
        # call constructor from superclass
        super().__init__()
        
        # define network layers
        self.fc1 = nn.Linear(6 * 30, 120)
        self.fc2 = nn.Linear(120, 100)
        self.fc3 = nn.Linear(100, 10000)
        
    def forward(self, x):
        # define forward pass
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# instantiate the model
model = Network()

# print model architecture
print(model)

# fully connected network from https://towardsdatascience.com/three-ways-to-build-a-neural-network-in-pytorch-8cea49f9a61a

Network(
  (fc1): Linear(in_features=180, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10000, bias=True)
)


In [91]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)

# from https://stackoverflow.com/questions/49433936/how-do-i-initialize-weights-in-pytorch

/var/folders/gz/pp4ldsnj0tjbngm2gygkp8qc0000gn/T/ipykernel_41421/3808141098.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Network(
  (fc1): Linear(in_features=180, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10000, bias=True)
)

In [92]:
loss_fn = torch.nn.CrossEntropyLoss()

# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [93]:
import random

def getTrainingData(n):
    for _ in range(n+1):
        n = random.randint(0, 9999)
        num = convert_to_words(str(n))
        # print(num)
        parts = num.split(" ")
        categories = ["zero", "one", "two", "three",
                     "four", "five", "six", "seven",
                     "eight", "nine", "ten", "eleven", "twelve",
                     "thirteen", "fourteen", "fifteen",
                     "sixteen", "seventeen", "eighteen",
                     "nineteen", "twenty", "thirty", "forty",
                     "fifty", "sixty", "seventy", "eighty",
                     "ninety", "hundred", "thousand"]
        values = []
        i = 0
        for part in parts:
            if part in categories and i < 6:
                values += [categories.index(part) + i * 30]
                i += 1
        x = torch.zeros(1, 180)
        ixs = torch.tensor(values)
        yield (x.index_add(1, ixs, torch.ones(1, len(values))), F.one_hot(torch.tensor([n]), 10000).double())

for d in getTrainingData(10):
    print(d[0].argmax().tolist(), d[1].argmax().tolist())

9 9917
9 9465
8 8465
5 5231
6 6641
9 9419
4 4054
5 5550
5 5130
6 6396
5 5235


In [94]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    datas = list(getTrainingData(9999))
    for i, data in enumerate(datas):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(datas) + i + 1
            running_loss = 0.

    return last_loss

    # from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [95]:
# Initializing in a separate cell so we can easily add more epochs to the same run
epoch_number = 0

EPOCHS = 500

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    print('LOSS train{}'.format(avg_loss))

    epoch_number += 1

# from https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

EPOCH 1:
  batch 1000 loss: 9.210403964042664
  batch 2000 loss: 9.210341051101684
  batch 3000 loss: 9.210375742912293
  batch 4000 loss: 9.210262008666993
  batch 5000 loss: 9.210280695915221
  batch 6000 loss: 9.210167917251587
  batch 7000 loss: 9.210229321479797
  batch 8000 loss: 9.210170796394348
  batch 9000 loss: 9.21015788078308
  batch 10000 loss: 9.210197590827942
LOSS train9.210197590827942
EPOCH 2:
  batch 1000 loss: 9.210007716178895
  batch 2000 loss: 9.21013210582733
  batch 3000 loss: 9.210206009864807
  batch 4000 loss: 9.209819379806518
  batch 5000 loss: 9.209968841552735
  batch 6000 loss: 9.210041770935058
  batch 7000 loss: 9.20993950366974
  batch 8000 loss: 9.209810066223145
  batch 9000 loss: 9.209686321258545
  batch 10000 loss: 9.209977289199829
LOSS train9.209977289199829
EPOCH 3:
  batch 1000 loss: 9.209915906906128
  batch 2000 loss: 9.209839703559876
  batch 3000 loss: 9.20986626625061
  batch 4000 loss: 9.20989029598236
  batch 5000 loss: 9.20964108371

KeyboardInterrupt: 

In [96]:
torch.save(model.state_dict(), "model")

In [97]:
saved_model = Network()
saved_model.load_state_dict(torch.load("model"))

<All keys matched successfully>

In [98]:
def predict(wordstr):
    parts = wordstr.split(" ")
    categories = ["zero", "one", "two", "three",
                    "four", "five", "six", "seven",
                    "eight", "nine", "ten", "eleven", "twelve",
                    "thirteen", "fourteen", "fifteen",
                    "sixteen", "seventeen", "eighteen",
                    "nineteen", "twenty", "thirty", "forty",
                    "fifty", "sixty", "seventy", "eighty",
                    "ninety", "hundred", "thousand"]
    values = []
    i = 0
    for part in parts:
        if part in categories and i < 6:
            values += [categories.index(part) + i * 30]
            i += 1
    x = torch.zeros(1, 180)
    ixs = torch.tensor(values)
    input = x.index_add(1, ixs, torch.ones(1, len(values)))
    return saved_model.forward(input)

In [104]:
predict("nine thousand one hundred and fifty").argmax().tolist()

9158

In [100]:
count = 0
for i in range(10000):
    if predict(convert_to_words(str(i))).argmax().tolist() == i:
        count += 1
print(count)

1448


In [53]:
count = 0
for i in range(10000):
    if random.randint(0,9999) == i:
        count += 1
print(count)

0


In [54]:
total = 0
for i in range(10000):
    total += abs(predict(convert_to_words(str(i))).argmax().tolist() - i)
print(total / 10000)

4330.8586


In [55]:
total = 0
for i in range(10000):
    total += abs(random.randint(0,9999) - i)
print(total / 10000)

3375.6591
